# Decoradores de clase

In [1]:
def typename(obj):
    return type(obj).__name__

In [2]:
class Localizacion:
    
    def __init__(self, nombre, posicion):
        
        self._nombre = nombre
        self._posicion = posicion
        
    @property
    def nombre(self):
        return self._nombre
    
    @property
    def posicion(self):
        return self._posicion
    
    def __repr__(self):
        return f"{typename(self)}(nombre={self.nombre}, posicion={self.posicion})"
    
    def __str__(self):
        return self.nombre

In [3]:
hong_kong = Localizacion("Hong Kong", (22.29, 114.16))
estocolmo = Localizacion("Estocolmo", (59.33, 18.06))
cape_tpwn = Localizacion("Cape Town", (-33.93, 18.42))
rotterdam = Localizacion("Rotterdam", (51.96, 4.47))
maracaibo = Localizacion("Maracaibo", (10.65, -71.65))

In [4]:
hong_kong

Localizacion(nombre=Hong Kong, posicion=(22.29, 114.16))

In [5]:
print(estocolmo)

Estocolmo


## Como sintetizar métodos

En lugar de tener el método repr() implementado, vamos a crear un decorador para dicha clase.

En dicho decorador vamos a mostrar todas las funciones miembro que se importan al importar dicha clase, junto a su correspondiente valor en memoria. Esto se hace gracias al método **vars() 

In [6]:
def auto_repr(cls):
    print(f"Decorando {cls.__name__} con autorepr")
    miembros = vars(cls)
    for nombre, miembro in miembros.items():
        print(nombre, miembro)
        
    return cls
    
@auto_repr
class Localizacion:
    
    def __init__(self, nombre, posicion):
        
        self._nombre = nombre
        self._posicion = posicion
        
    @property
    def nombre(self):
        return self._nombre
    
    @property
    def posicion(self):
        return self._posicion
    
    def __str__(self):
        return self.nombre

Decorando Localizacion con autorepr
__module__ __main__
__init__ <function Localizacion.__init__ at 0x00000254C8D88A60>
nombre <property object at 0x00000254C8D6DB80>
posicion <property object at 0x00000254C8D9D720>
__str__ <function Localizacion.__str__ at 0x00000254C8D88C10>
__dict__ <attribute '__dict__' of 'Localizacion' objects>
__weakref__ <attribute '__weakref__' of 'Localizacion' objects>
__doc__ None


Como podemos ver, aparecen:
    
- El inicializador de la clase.
- Las dos properties: nombre y posicion.
- El método \__str\__
    
El resto de métodos que aparecen corresponden al funcionamiento interno de Python.



Para ver si la definición de nuestra clase contiene la función \__repr\__, podemos hacer una comprobación dentro del decorador:

In [7]:
def auto_repr(cls):
    
    print(f"Decorando {cls.__name__} con autorepr")
    
    miembros = vars(cls)
    
    for nombre, miembro in miembros.items():
        print(nombre, miembro)
    
    if "__repr__" in miembros:
        raise TypeError(f"La clase {cls.__name__} ya define el propio __repr__")
    
        
    return cls

Podemos hacer lo mismo para asegurarnos de que se define un inicializador:

In [8]:
def auto_repr(cls):
    
    print(f"Decorando {cls.__name__} con autorepr")
    
    miembros = vars(cls)
    
    for nombre, miembro in miembros.items():
        print(nombre, miembro)
    
    if "__repr__" in miembros:
        raise TypeError(f"La clase {cls.__name__} ya define el propio __repr__")
    
    if "__init__" not in miembros:
        raise TypeError(f"La clase {cls.__name__} debe tener un método __init__")
    
    return cls

La proxima comprobación que vamos a hacer es un poco más compleja. 

Vamos a comprobar que para cada argumento de __init__ aparte de **self**, existe una property definida.

Podemos obtener la lista de argumentos existente para \__init\__ pasándole una referencia del método a la función **signature**:

In [9]:
import inspect

In [10]:
def auto_repr(cls):
    
    print(f"Decorando {cls.__name__} con autorepr")
    
    miembros = vars(cls)
    
    for nombre, miembro in miembros.items():
        print(nombre, miembro)
    
    if "__repr__" in miembros:
        raise TypeError(f"La clase {cls.__name__} ya define el propio __repr__")
    
    if "__init__" not in miembros:
        raise TypeError(f"La clase {cls.__name__} debe tener un método __init__")
        
    sig = inspect.signature(cls.__init__)
    
    parametros = list(sig.parameters)[1:]
    
    print("Parámetros de __init__: ", parametros)
    
    # Lo que hacemos aquí es decirle: 
    # 1. Para cada property, intenta obtenerla del diccionario miembros
    # 2. Si no está, devuelve None
    # Si para alguno de los valores devuelve None, es que ese valor no tiene un property establecido
    
    if not all(isinstance(miembros.get(nombre, None), property) for nombre in parametros):
        raise TypeError("Todos los argumentos de __init__ deben tener una property establecida.")
    
    return cls

Ahora, vamos a crear el decorador de clase el método repr() "sintetizado":

In [11]:
def repr_sintetizado(self):
    
    return "{typename}({args})".format(
        typename=typename(self),
        args=", ".join(
            "{nombre}={valor!r}".format(
                nombre=nombre,
                valor=getattr(self, nombre)
            ) for nombre in parametros
        )
    )

    setattr(cls, "__repr__", repr_sintetizado)

Y lo incluimos en el decorador:

In [12]:
def auto_repr(cls):
    
    miembros = vars(cls)
    
    for nombre, miembro in miembros.items():
        print(nombre, miembro)
    
    if "__repr__" in miembros:
        raise TypeError(f"La clase {cls.__name__} ya define el propio __repr__")
    
    if "__init__" not in miembros:
        raise TypeError(f"La clase {cls.__name__} debe tener un método __init__")
        
    sig = inspect.signature(cls.__init__)
    
    parametros = list(sig.parameters)[1:]
    
    # Lo que hacemos aquí es decirle: 
    # 1. Para cada property, intenta obtenerla del diccionario miembros
    # 2. Si no está, devuelve None
    # Si para alguno de los valores devuelve None, es que ese valor no tiene un property establecido
    
    if not all(isinstance(miembros.get(nombre, None), property) for nombre in parametros):
        raise TypeError("Todos los argumentos de __init__ deben tener una property establecida.")
    
    def repr_sintetizado(self):
    
        return "{typename}({args})".format(
            typename=typename(self),
            args=", ".join(
                "{nombre}={valor}".format(
                    nombre=nombre,
                    valor=getattr(self, nombre)
                ) for nombre in parametros
            )
        )

    setattr(cls, "__repr__", repr_sintetizado)
    
    return cls

Si ahora definimos la clase con el decorador:

In [13]:
@auto_repr
class Localizacion:
    
    def __init__(self, nombre, posicion):
        
        self._nombre = nombre
        self._posicion = posicion
        
    @property
    def nombre(self):
        return self._nombre
    
    @property
    def posicion(self):
        return self._posicion
    
    def __str__(self):
        return self.nombre

__module__ __main__
__init__ <function Localizacion.__init__ at 0x00000254C8D96A60>
nombre <property object at 0x00000254C8DA0B80>
posicion <property object at 0x00000254C8DA0DB0>
__str__ <function Localizacion.__str__ at 0x00000254C8D96C10>
__dict__ <attribute '__dict__' of 'Localizacion' objects>
__weakref__ <attribute '__weakref__' of 'Localizacion' objects>
__doc__ None


Y creamos una instancia de la clase:

In [14]:
hong_kong = Localizacion("Hong Kong", (22.29, 114.16))

In [15]:
hong_kong

Localizacion(nombre=Hong Kong, posicion=(22.29, 114.16))

Vemos como __repr__ funciona perfectamente.

## Resumen

En lo que ha consistido este notebook es basicamente en meter la función __repr__ en el decorador y asignarsela a la clase a través del mismo.